# Description
- This ipynb file is used to run live trading.
- Users proceed with live trading based on strategies that have been verified and successfully backtested.

1. Load API Key
2. Upload to Main Server
   - Just like in backtesting, upload both the strategy and config files to the server.  
2-1. Upload strategy  
2-2. Upload config  

3. DELETE – Optional
4. Run Auto-Trade
   - Download the log_viewer.py file
   - Run the #run-system cell
   - Then, execute the terminal command to run log_viewer.py
5. Terminate
6. Confirm Position Holding After Terminating
 
 
#### - Original GitHub Link : https://github.com/NeoMatrixAI/nb-runner/tree/main/notebooks 

# Load Api Key

In [20]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='./module/.env')

USER_KEY = os.getenv("USER_KEY")

print("USER_KEY:", USER_KEY if USER_KEY else "Not found")

# health check

In [21]:
import requests

root_url = f'https://aifapbt.fin.cloud.ainode.ai/{USER_KEY}/'
requests.get(root_url).json()

# upload to main server

## strategy

In [ ]:
strategy_name = "multi_period_momentum"

In [ ]:
# Upload strategy file
endpoint = 'upload/strategy/'
url = root_url + endpoint

file_path = strategy_name + ".py"

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, files=files)

print("📂 Upload Response:", response.json())

In [24]:
# Check strategy file upload
endpoint = 'upload/check/strategy/'
url = root_url + endpoint

params = {"strategy_name": strategy_name}

response = requests.get(url, params=params)
print(response.json()['content'])

In [25]:
# Check all uploaded strategy files
endpoint = 'upload/check/all-files/strategy/'
url = root_url + endpoint

response = requests.get(url)
print(response.json())

## DELETE

In [7]:
# # Delete uploaded strategy settings file
# endpoint = 'upload/delete/strategy/'
# url = root_url + endpoint

# params = {"strategy_name": strategy_name}

# # Sending a DELETE request
# response = requests.delete(url, params=params)
# print(response.json())

## config

In [26]:
# Upload strategy file
endpoint = 'upload/config/'
url = root_url + endpoint

config_name = strategy_name + "_config"
file_path = config_name + ".py"

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, files=files)

print("📂 Upload Response:", response.json())

In [ ]:
# Check strategy file upload
endpoint = 'upload/check/config/'
url = root_url + endpoint

params = {"config_name": config_name}

response = requests.get(url, params=params)
print(response.json()['content'])

In [28]:
# Check all uploaded config files
endpoint = 'upload/check/all-files/config/'
url = root_url + endpoint

response = requests.get(url)
print(response.json())

## DELETE

In [1]:
# # Delete uploaded strategy settings file
# endpoint = 'upload/delete/config/'
# url = root_url + endpoint

# params = {"config_name": config_name}

# # Sending a DELETE request
# response = requests.delete(url, params=params)
# print(response.json())

# Run Auto-Trade

In [29]:
# download oncetime
!wget https://raw.githubusercontent.com/NeoMatrixAI/nb-runner/main/modules/log_viewer.py # trade log module

In [30]:
# run-system
endpoint = 'command/run-system'
url = root_url + endpoint

data = {
    "strategy_name": strategy_name,
    "method": "rebalancing"
}

response = requests.post(url, json=data)

try:
  session_id = response.json()['session_id']
  print('session id :',response.json()['session_id'])
  print('dashboard :',response.json()['dashboard_url'])
except:
  print('Error :',response.json()['message']['message'])

  session_id = response.json()['message']['session_id']
  print('session id :',session_id)

  print('dashboard :',response.json()['message']['dashboard_url'])

In [ ]:
# !python {your path for 'log_viewer.py'} --USER_KEY {USER_KEY} --session_id {session_id} --save_log true/false
!python /content/drive/MyDrive/log_viewer.py --USER_KEY {USER_KEY} --session_id {session_id} --save_log true

# terminate

In [32]:
# terminate process
# You must execute terminate to force liquidation of the current process.
endpoint = 'command/terminate/'
url = root_url + endpoint

params = {"session_id": f"{session_id}"}

response = requests.get(url, params=params)
print(response.json())

# Confirm position holding after Terminating

In [ ]:
import pandas as pd

def all_positions(user_key, productType, marginCoin):
    url = f"https://bitgettrader.fin.cloud.ainode.ai/{USER_KEY}/future/position/all-positions"

    payload = {
        "user_key": USER_KEY,
        "all_positions": {
            "productType": productType,
            'marginCoin': marginCoin.upper()
        }
    }

    response = requests.post(url, json=payload)
    data = response.json()['data']
    df = pd.DataFrame(data)
    if df.empty:
        print('Everything has been liquidated')
        pass
    return df


productType = 'susdt-futures
marginCoin = 'susdt'
df = all_positions(USER_KEY, productType, marginCoin)
df